In [9]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")

import MetaTrader5 as mt
from datetime import datetime as dt

# custom
from my_jupyter.user import User
from my_jupyter.strategy_manager import StrategyManager
from my_jupyter.operation import Operation
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.market_data_repository import MarketDataRepository
from my_jupyter.filters.moving_average import MovingAverageFilter
from my_jupyter.filters.consequent_limits_counter import ConsequentLimitsCounterFilter
from my_jupyter.filters.directioned_bars_filter import DirectionedBarsFilter

# from Mbox import Mbox
# from boleta_1 import gerar_boleta
# from gerenciamento import Gerenciamento

## GUI's
from daemon_tools import DaemonThreadMini


ModuleNotFoundError: No module named 'MetaTrader5'

In [ ]:


filters = [DirectionedBarsFilter(candles=5), MovingAverageFilter(), MovingAverageFilter(period=200)]
strategies = [MinMaxStrategy(filters=filters)]
operations = [Operation("WINV23", 1, mt.TIMEFRAME_M1, can_buy=True, can_sell=True)]
user = User(strategies=strategies, operations=operations, trading_time_start=dt(2023,10,5,9,5,0))
market_data_repository = MarketDataRepository()
strategy = StrategyManager(market_data_repository, user)

# PARAMETROS MAIN
kwargs = {
    "strategy": strategy,
}


def main(**kwargs):
    _shift = 0
    strategy: StrategyManager = kwargs["strategy"]
    strategy.run_strategies()
    # print(f"{dt.now().strftime('%d/%m/%Y, %H:%M:%S')} >>> D-{_shift}")
def exec_daemon(**kwargs):
    main(**kwargs)
    # alarme.ring_alarme()   

In [ ]:
# gerenciamento = Gerenciamento(mt, timeframe, timeframes_dict)

try:
    main(**kwargs)
except Exception as e:
    print(e.with_traceback(__tb=None))
    # Mbox.Alerta("ERRO", "busca de sinais com erro")


period 2
ohlc [(1696528860, 113750., 113750., 113750., 113750.,   10, 0,  8642)
 (1696528440, 113790., 113805., 113750., 113795.,  337, 5,  4022)
 (1696528380, 113790., 113815., 113765., 113790.,  634, 5,  9666)
 (1696528320, 113790., 113805., 113780., 113800.,  280, 5,  3999)
 (1696528260, 113805., 113810., 113785., 113790.,  346, 5,  3703)
 (1696528200, 113765., 113815., 113765., 113800.,  810, 5,  8626)
 (1696528140, 113750., 113780., 113725., 113770.,  870, 5, 15528)
 (1696528080, 113775., 113780., 113745., 113750.,  565, 5,  6308)
 (1696528020, 113780., 113785., 113760., 113775.,  451, 5,  5814)
 (1696527960, 113770., 113795., 113765., 113780.,  861, 5,  8698)
 (1696527900, 113750., 113785., 113740., 113770., 1304, 5, 14311)
 (1696527840, 113740., 113760., 113730., 113750.,  908, 5, 12704)
 (1696527780, 113720., 113775., 113715., 113740., 1655, 5, 22872)
 (1696527720, 113635., 113725., 113635., 113715., 1958, 5, 33174)
 (1696527660, 113615., 113645., 113615., 113635.,  431, 5,  55

TypeError: BaseException.with_traceback() takes no keyword arguments

In [ ]:
# from my_jupyter.daemons.daemon_tools import DaemonThreadMini


# thread_exec = DaemonThreadMini(exec_daemon, seconds=0, mseconds=800, **kwargs)
# thread_exec.start()

In [2]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")
# print(*sys.path, sep="\n")

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG
import pandas as pd
# custom
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.filters.moving_average import MovingAverageFilter
from my_jupyter.filters.directioned_bars_filter import DirectionedBarsFilter
filters = [DirectionedBarsFilter(candles=5), MovingAverageFilter(), MovingAverageFilter(period=200)]


def _read_file(filename):
    from os.path import dirname, join, abspath

    return pd.read_csv(join(abspath('.'), filename),
                       index_col=0, parse_dates=True, infer_datetime_format=True)
WINV23M1 = _read_file('dataset\WINV23M1 copy.csv')

def data(arr):
    return arr.rename(columns={'Close': 'close','High': 'high', 'Low':'low', 'Open':'open'})

class SmaCross(Strategy):
    def init(self):
        self.min_max = MinMaxStrategy()
        self.ohlc = self.I(data, self.data.df)
        # self.data.df = self.data.df.rename(columns={'Close': 'close','High': 'high', 'Low':'low', 'Open':'open'})

    def next(self):
        ohlc = self.ohlc #.rename(columns={'Close': 'close','High': 'high', 'Low':'low', 'Open':'open'})
        ohlc={
            'open': list(ohlc[0][::-1]),
            'high': list(ohlc[1][::-1]),
            'low': list(ohlc[2][::-1]),
            'close': list(ohlc[3][::-1]),
        }
        ohlc = pd.DataFrame.from_dict(ohlc)[::-1]
        while len(ohlc) < self.min_max.candles_needed:
            return 
        if self.position.is_long:
            if self.min_max.buy_close(ohlc):
                self.position.close()
            return
        elif self.position.is_short:
            if self.min_max.sell_close(ohlc):
                self.position.close()
            return

        if self.min_max.check_buy_signal(ohlc):
            a = self.buy()
        elif self.min_max.check_sell_signal(ohlc):
            a = self.sell()


bt = Backtest(WINV23M1, SmaCross, commission=0.002, exclusive_orders=True, cash=10**12)
stats = bt.run()
bt.plot()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_72436\1483159935.py:21: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(abspath('.'), filename),


sell
<Order size=-1.0, contingent=0>
False 113975.0 113665.0
False 113980.0 113665.0
False 113955.0 113665.0
False 113895.0 113665.0
False 113900.0 113665.0
False 113855.0 113665.0
False 113860.0 113665.0
False 113815.0 113665.0
False 113730.0 113665.0
False 113690.0 113665.0
False 113720.0 113665.0
False 113755.0 113665.0
False 113730.0 113665.0
False 113735.0 113665.0
False 113720.0 113665.0
False 113685.0 113665.0
False 113705.0 113665.0
False 113710.0 113665.0
False 113735.0 113665.0
False 113685.0 113665.0
False 113695.0 113665.0
False 113675.0 113665.0
False 113690.0 113665.0
True 113660.0 113665.0
buy
<Order size=1.0, contingent=0>
sell
<Order size=-1.0, contingent=0>
False 114035.0 113665.0
False 114065.0 113665.0
False 114060.0 113665.0
False 113965.0 113665.0
False 114000.0 113665.0
False 114040.0 113665.0
False 114045.0 113665.0
False 114075.0 113665.0
False 114085.0 113665.0
False 114165.0 113665.0
False 114255.0 113665.0
False 114230.0 113665.0
False 114205.0 113665.0
Fals

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p1724', ...)

In [2]:
stats

Start                     2023-10-05 09:00:00
End                       2023-10-05 18:01:00
Duration                      0 days 09:01:00
Exposure Time [%]                   98.134328
Equity Final [$]          999999903461.879883
Equity Peak [$]               1000000000000.0
Return [%]                           -0.00001
Buy & Hold Return [%]               -0.079058
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                    -0.00001
Avg. Drawdown [%]                    -0.00001
Max. Drawdown Duration        0 days 08:52:00
Avg. Drawdown Duration        0 days 08:52:00
# Trades                                  430
Win Rate [%]                         0.697674
Best Trade [%]                       0.190372
Worst Trade [%]                     -0.345085
Avg. Trade [%]                    